# **Adversarial Robustness of VGG-16, ResNet18, and EfficientNet-B0 on CIFAR-10**
# **Pretrain=False, 10 epoch**

## **1. Import Required Libraries**

In [1]:
import torch
print(torch.cuda.is_available())  # Should return True if GPU is available
print(torch.cuda.get_device_name(0))  # Should print the name of your GPU

True
NVIDIA GeForce RTX 4060


In [4]:

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torchattacks
import time
import pandas as pd
from torchvision import models

# Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')


Using device: cuda


## **2. Load CIFAR-10 Dataset**

In [5]:

# Load CIFAR-10 dataset
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize for model compatibility
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=False)


100%|███████████████████████████████████████████████████████████████████████████████| 170M/170M [00:23<00:00, 7.21MB/s]


# **3. Defining Models From torchvision**

## **3.1 Define VGG-16 Model**

In [6]:

# Load pre-defined VGG-16 model from torchvision
class VGG16(nn.Module):
    def __init__(self):
        super(VGG16, self).__init__()
        self.model = models.vgg16(pretrained=False)
        num_ftrs = self.model.classifier[6].in_features
        self.model.classifier[6] = nn.Linear(num_ftrs, 10)  # Adjust for CIFAR-10 classes

    def forward(self, x):
        return self.model(x)

# Initialize model
model_vgg = VGG16().to(device)
criterion = nn.CrossEntropyLoss()
optimizer_vgg = optim.Adam(model_vgg.parameters(), lr=0.001)


C:\Users\laika\AppData\Local\Programs\Python\Python313\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\laika\AppData\Local\Programs\Python\Python313\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


## **3.2 Define ResNet18 Model**

In [7]:

# Load pre-defined ResNet18 model from torchvision
class ResNet18(nn.Module):
    def __init__(self):
        super(ResNet18, self).__init__()
        self.model = models.resnet18(pretrained=False)
        num_ftrs = self.model.fc.in_features
        self.model.fc = nn.Linear(num_ftrs, 10)  # Adjust for CIFAR-10 classes

    def forward(self, x):
        return self.model(x)

# Initialize model
model_resnet = ResNet18().to(device)
optimizer_resnet = optim.Adam(model_resnet.parameters(), lr=0.001)


## **3.3 Define EfficientNet-B0 Model**

In [8]:

# Load pre-defined EfficientNet-B0 model from torchvision
class EfficientNetB0(nn.Module):
    def __init__(self):
        super(EfficientNetB0, self).__init__()
        self.model = models.efficientnet_b0(pretrained=False)
        num_ftrs = self.model.classifier[1].in_features
        self.model.classifier[1] = nn.Linear(num_ftrs, 10)  # Adjust for CIFAR-10 classes

    def forward(self, x):
        return self.model(x)

# Initialize model
model_efficient = EfficientNetB0().to(device)
optimizer_efficient = optim.Adam(model_efficient.parameters(), lr=0.001)


# **4. Train the Models**

In [9]:

# Training function
def train_model(model, optimizer, model_name, epochs=10):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for images, labels in trainloader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        print(f"{model_name} - Epoch {epoch+1}, Loss: {running_loss/len(trainloader):.4f}")

print("Training VGG-16...")
train_model(model_vgg, optimizer_vgg, "VGG-16")

print("Training ResNet18...")
train_model(model_resnet, optimizer_resnet, "ResNet18")

print("Training EfficientNet-B0...")
train_model(model_efficient, optimizer_efficient, "EfficientNet-B0")


Training VGG-16...
VGG-16 - Epoch 1, Loss: 2.4705
VGG-16 - Epoch 2, Loss: 2.3029
VGG-16 - Epoch 3, Loss: 2.3028
VGG-16 - Epoch 4, Loss: 2.3028
VGG-16 - Epoch 5, Loss: 2.3028
VGG-16 - Epoch 6, Loss: 2.3028
VGG-16 - Epoch 7, Loss: 2.3028
VGG-16 - Epoch 8, Loss: 2.3028
VGG-16 - Epoch 9, Loss: 2.3028
VGG-16 - Epoch 10, Loss: 2.3028
Training ResNet18...
ResNet18 - Epoch 1, Loss: 1.3345
ResNet18 - Epoch 2, Loss: 0.8050
ResNet18 - Epoch 3, Loss: 0.6128
ResNet18 - Epoch 4, Loss: 0.4907
ResNet18 - Epoch 5, Loss: 0.3834
ResNet18 - Epoch 6, Loss: 0.2876
ResNet18 - Epoch 7, Loss: 0.2106
ResNet18 - Epoch 8, Loss: 0.1491
ResNet18 - Epoch 9, Loss: 0.1171
ResNet18 - Epoch 10, Loss: 0.0819
Training EfficientNet-B0...
EfficientNet-B0 - Epoch 1, Loss: 1.4905
EfficientNet-B0 - Epoch 2, Loss: 0.9091
EfficientNet-B0 - Epoch 3, Loss: 0.6575
EfficientNet-B0 - Epoch 4, Loss: 0.5278
EfficientNet-B0 - Epoch 5, Loss: 0.4445
EfficientNet-B0 - Epoch 6, Loss: 0.3807
EfficientNet-B0 - Epoch 7, Loss: 0.3203
EfficientN

# **5. Evaluate Model Accuracy on Clean Test Images**

In [10]:

# Evaluation function
def test_model(model, dataloader, model_name):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'{model_name} Accuracy on clean test images: {accuracy:.2f}%')
    return accuracy

clean_accuracy_vgg = test_model(model_vgg, testloader, "VGG-16")
clean_accuracy_resnet = test_model(model_resnet, testloader, "ResNet18")
clean_accuracy_efficient = test_model(model_efficient, testloader, "EfficientNet-B0")


VGG-16 Accuracy on clean test images: 10.00%
ResNet18 Accuracy on clean test images: 84.25%
EfficientNet-B0 Accuracy on clean test images: 86.60%


# **6. Setting up for evaluating Adversarial attacks**

In [12]:
# Function to evaluate adversarial attacks
def test_adversarial_attack(model, attack, attack_name):
    model.eval()
    correct = 0
    total = 0
    success_count = 0
    start_time = time.time()

    for images, labels in testloader:
        images, labels = images.to(device), labels.to(device)
        adv_images = attack(images, labels)
        outputs = model(adv_images)
        _, predicted = torch.max(outputs.data, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        success_count += (predicted != labels).sum().item()

    end_time = time.time()
    attack_time = end_time - start_time
    attack_accuracy = 100 * correct / total
    attack_success_rate = 100 * success_count / total

    print(f'{attack_name} Attack Results:')
    print(f' - Accuracy after attack: {attack_accuracy:.2f}%')
    print(f' - Attack Success Rate: {attack_success_rate:.2f}%')
    print(f' - Time Taken: {attack_time:.2f} seconds\n')

    return attack_accuracy, attack_success_rate, attack_time

# **6.1 Evaluating Adversarial Robustness of VGG-16**

In [11]:
# Define adversarial attacks
attacks = {
    "FGSM": torchattacks.FGSM(model_vgg, eps=0.03),
    "PGD": torchattacks.PGD(model_vgg, eps=0.03, alpha=0.01, steps=40),
    "DeepFool": torchattacks.DeepFool(model_vgg, steps=50),
    "BIM": torchattacks.BIM(model_vgg, eps=0.03, alpha=0.01, steps=10),
    "C&W": torchattacks.CW(model_vgg, c=1, kappa=0, steps=100, lr=0.01)
}

In [13]:
# Run attacks and collect results
attack_results = {}

for attack_name, attack in attacks.items():
    attack_results[attack_name] = test_adversarial_attack(model_vgg, attack, attack_name)

# Print summary
print("\nFinal Adversarial Attack Summary:")
for attack, (acc, success_rate, time_taken) in attack_results.items():
    print(f"{attack}:")
    print(f" - Accuracy After Attack: {acc:.2f}%")
    print(f" - Attack Success Rate: {success_rate:.2f}%")
    print(f" - Time Taken: {time_taken:.2f} sec\n")

FGSM Attack Results:
 - Accuracy after attack: 10.00%
 - Attack Success Rate: 90.00%
 - Time Taken: 160.22 seconds

PGD Attack Results:
 - Accuracy after attack: 10.00%
 - Attack Success Rate: 90.00%
 - Time Taken: 11968.83 seconds

DeepFool Attack Results:
 - Accuracy after attack: 0.00%
 - Attack Success Rate: 100.00%
 - Time Taken: 628.24 seconds

BIM Attack Results:
 - Accuracy after attack: 10.00%
 - Attack Success Rate: 90.00%
 - Time Taken: 1357.61 seconds

C&W Attack Results:
 - Accuracy after attack: 10.00%
 - Attack Success Rate: 90.00%
 - Time Taken: 27703.64 seconds


Final Adversarial Attack Summary:
FGSM:
 - Accuracy After Attack: 10.00%
 - Attack Success Rate: 90.00%
 - Time Taken: 160.22 sec

PGD:
 - Accuracy After Attack: 10.00%
 - Attack Success Rate: 90.00%
 - Time Taken: 11968.83 sec

DeepFool:
 - Accuracy After Attack: 0.00%
 - Attack Success Rate: 100.00%
 - Time Taken: 628.24 sec

BIM:
 - Accuracy After Attack: 10.00%
 - Attack Success Rate: 90.00%
 - Time Taken:

# **6.2 Evaluating Adversarial Robustness of ResNet18**

In [19]:
# Define adversarial attacks
attacks = {
    "FGSM": torchattacks.FGSM(model_resnet, eps=0.03),
    "PGD": torchattacks.PGD(model_resnet, eps=0.03, alpha=0.01, steps=40),
    "DeepFool": torchattacks.DeepFool(model_resnet, steps=50),
    "BIM": torchattacks.BIM(model_resnet, eps=0.03, alpha=0.01, steps=10),
    "C&W": torchattacks.CW(model_resnet, c=1, kappa=0, steps=100, lr=0.01)
}

In [20]:
# ReRun Resnet attacks and collect results
attack_results = {}

for attack_name, attack in attacks.items():
    attack_results[attack_name] = test_adversarial_attack(model_resnet, attack, attack_name)

# Print summary
print("\nFinal Adversarial Attack Summary on ResNet18:")
for attack, (acc, success_rate, time_taken) in attack_results.items():
    print(f"{attack}:")
    print(f" - Accuracy After Attack: {acc:.2f}%")
    print(f" - Attack Success Rate: {success_rate:.2f}%")
    print(f" - Time Taken: {time_taken:.2f} sec\n")

FGSM Attack Results:
 - Accuracy after attack: 10.66%
 - Attack Success Rate: 89.34%
 - Time Taken: 23.85 seconds

PGD Attack Results:
 - Accuracy after attack: 0.34%
 - Attack Success Rate: 99.66%
 - Time Taken: 527.25 seconds

DeepFool Attack Results:
 - Accuracy after attack: 14.37%
 - Attack Success Rate: 85.63%
 - Time Taken: 29661.24 seconds

BIM Attack Results:
 - Accuracy after attack: 6.83%
 - Attack Success Rate: 93.17%
 - Time Taken: 144.56 seconds

C&W Attack Results:
 - Accuracy after attack: 2.14%
 - Attack Success Rate: 97.86%
 - Time Taken: 1142.75 seconds


Final Adversarial Attack Summary on ResNet18:
FGSM:
 - Accuracy After Attack: 10.66%
 - Attack Success Rate: 89.34%
 - Time Taken: 23.85 sec

PGD:
 - Accuracy After Attack: 0.34%
 - Attack Success Rate: 99.66%
 - Time Taken: 527.25 sec

DeepFool:
 - Accuracy After Attack: 14.37%
 - Attack Success Rate: 85.63%
 - Time Taken: 29661.24 sec

BIM:
 - Accuracy After Attack: 6.83%
 - Attack Success Rate: 93.17%
 - Time Tak

# **6.3 Evaluating Adversarial Robustness of EfficientNet-B0**

In [21]:
# Define adversarial attacks
attacks = {
    "FGSM": torchattacks.FGSM(model_efficient, eps=0.03),
    "PGD": torchattacks.PGD(model_efficient, eps=0.03, alpha=0.01, steps=40),
    "DeepFool": torchattacks.DeepFool(model_efficient, steps=50),
    "BIM": torchattacks.BIM(model_efficient, eps=0.03, alpha=0.01, steps=10),
    "C&W": torchattacks.CW(model_efficient, c=1, kappa=0, steps=100, lr=0.01)
}

In [22]:
# ReRun attacks and collect results
attack_results = {}

for attack_name, attack in attacks.items():
    attack_results[attack_name] = test_adversarial_attack(model_efficient, attack, attack_name)

# Print summary
print("\nFinal Adversarial Attack Summary on EfficientNet-B0:")
for attack, (acc, success_rate, time_taken) in attack_results.items():
    print(f"{attack}:")
    print(f" - Accuracy After Attack: {acc:.2f}%")
    print(f" - Attack Success Rate: {success_rate:.2f}%")
    print(f" - Time Taken: {time_taken:.2f} sec\n")

FGSM Attack Results:
 - Accuracy after attack: 12.84%
 - Attack Success Rate: 87.16%
 - Time Taken: 231.91 seconds

PGD Attack Results:
 - Accuracy after attack: 0.00%
 - Attack Success Rate: 100.00%
 - Time Taken: 6407.64 seconds

DeepFool Attack Results:
 - Accuracy after attack: 14.09%
 - Attack Success Rate: 85.91%
 - Time Taken: 6539.50 seconds

BIM Attack Results:
 - Accuracy after attack: 2.55%
 - Attack Success Rate: 97.45%
 - Time Taken: 1818.97 seconds

C&W Attack Results:
 - Accuracy after attack: 0.01%
 - Attack Success Rate: 99.99%
 - Time Taken: 14381.82 seconds


Final Adversarial Attack Summary on EfficientNet-B0:
FGSM:
 - Accuracy After Attack: 12.84%
 - Attack Success Rate: 87.16%
 - Time Taken: 231.91 sec

PGD:
 - Accuracy After Attack: 0.00%
 - Attack Success Rate: 100.00%
 - Time Taken: 6407.64 sec

DeepFool:
 - Accuracy After Attack: 14.09%
 - Attack Success Rate: 85.91%
 - Time Taken: 6539.50 sec

BIM:
 - Accuracy After Attack: 2.55%
 - Attack Success Rate: 97.45